In [1]:
from IPython.display import Image

## dtypes

In [4]:
# fp32
# tf32: 没有32位
# fp16
# bf16
Image(url='../../imgs/tf32.jpeg', width=400)

## AMP (automatic mixed precision)

```
import torch
from torch.cuda.amp import autocast, GradScaler

# 创建模型和优化器
model = ...
optimizer = ...

# 创建 GradScaler
scaler = GradScaler()

for data, target in dataloader:
    optimizer.zero_grad()

    # 在前向传播中启用 autocast
    with autocast():
        output = model(data)
        loss = loss_function(output, target)

    # 反向传播和优化步骤
    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()
```


- 关键操作使用 fp32；
    - 保留关键操作（如梯度累积和权重更新）使用 FP32，这些操作对训练稳定性和精度非常重要。
    - 非关键操作（如前向传播中的大部分计算）使用 FP16，提高了计算效率和内存利用率。

- 动态损失缩放：
    - 动态损失缩放技术通过缩放损失值，确保在 FP16 中进行计算时不会出现溢出或下溢问题。
    - 这有助于在训练过程中保持数值稳定，减少由于精度不足导致的训练问题。


- 损失缩放（loss scaling）: `scaler.scale(loss)`
    - `*scale (1024)`
- 反向传播计算梯度：`(loss * 1024).backward()`
- 梯度缩放还原：
    - 在 scaler.step(optimizer) 中，会将放大的梯度还原到原始范围，即除以 1024。
        - 具体来说，scaler.step(optimizer) 会对每个参数的梯度进行缩放还原，`param.grad = param.grad / 1024`，从而恢复梯度的实际大小。
- 动态调整缩放因子：
    - 在 scaler.update() 中，会检查是否在前面的步骤中发生了数值溢出。
    - 如果检测到数值溢出，缩放因子会减小（例如乘以 0.5）。
    - 如果没有检测到溢出，缩放因子可能会增大（例如乘以 2），以优化计算性能。